In [3]:
import os
import sentencepiece as spm

input_file = "data/petitions_splited_mecab.txt"
vocab_size = 23000

sp_model_root  = "data/sentencepiece"

if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)

sp_model_name        = 'tokenizer_%d'
sp_model_path        = os.path.join(sp_model_root, sp_model_name)
model_type           = 'unigram'
character_coverage   = 1.0
user_defined_symbols = '[BOS],[EOS]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_file, sp_model_path, vocab_size, user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)

In [4]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(sp_model_path))

tokens = sp.encode_as_pieces('나는 오늘 아침밥을 먹었다.')
ids = sp.encode_as_ids('나는 오늘 아침밥을 먹었다.')

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[67, 391, 792, 1541, 4357, 886, 236, 1697, 326, 929]
['▁나', '는', '▁오늘', '▁아침', '밥', '을', '▁먹', '었', '다', '.']
나는 오늘 아침밥을 먹었다.
나는 오늘 아침밥을 먹었다.


In [5]:
import itertools
from konlpy.tag import Mecab
from tqdm import tqdm

# mecab = Mecab().morphs
mecab = Mecab()

with open("data/petitions_2019-01", 'r') as f:
    data = f.readlines()
    
sublist   = [line.split('"content": "')[-1].split('", "num_agree": ')[0] for line in data]
sublist   = [doc.split(". ") for doc in sublist]
petitions = list(itertools.chain.from_iterable(sublist))
petitions[:10]


['국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다',
 '기해년 새해 복 많이 받으십시오',
 '저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고 합니다',
 '저는 8기의 원전이 가동․건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 원전지역 주민입니다',
 '간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다',
 '‘울진군민과 약속한 신한울 3,4호기 원전건설을 재개해 주십시오.’ 여태껏 단 한 번도 원전 건설을 원한 적 없는 제가 신한울 3,4호기 원전 건설을 청하는 까닭을 말씀드리겠습니다',
 '경상북도 동해안 최북단 울진군은 예부터 산과 바다, 계곡의 울창함이 보배처럼 아름답다하여 “울진(蔚珍)”이라는 지명을 간직하게 된 곳입니다',
 '이러한 곳에 1981년 원전사업의 시작으로 울진군에 북면(6기), 산포지구(6기), 직산지구(6기)가 원전 예정지역으로 지정되면서, 먼저 북면 부구리 지역에 원전 6기가 건설되었습니다',
 '해안선이 잘려나가고 마을 한복판에 고압 송전탑이 들어섰습니다',
 '어장이 파괴되고 지역 특산품에 방사능 꼬리표가 붙었습니다']

In [18]:
with open("data/petitions_2019-01", 'r') as f:
    data = f.readlines()
    
sublist   = [line.split('"content": "')[-1].split('", "num_agree": ')[0] for line in data]
sublist   = [doc.split(". ") for doc in sublist]
petitions = list(itertools.chain.from_iterable(sublist))
petitions = [petition + ".\n" for petition in petitions]
petitions

['국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다.\n',
 '기해년 새해 복 많이 받으십시오.\n',
 '저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고 합니다.\n',
 '저는 8기의 원전이 가동․건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 원전지역 주민입니다.\n',
 '간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다.\n',
 '‘울진군민과 약속한 신한울 3,4호기 원전건설을 재개해 주십시오.’ 여태껏 단 한 번도 원전 건설을 원한 적 없는 제가 신한울 3,4호기 원전 건설을 청하는 까닭을 말씀드리겠습니다.\n',
 '경상북도 동해안 최북단 울진군은 예부터 산과 바다, 계곡의 울창함이 보배처럼 아름답다하여 “울진(蔚珍)”이라는 지명을 간직하게 된 곳입니다.\n',
 '이러한 곳에 1981년 원전사업의 시작으로 울진군에 북면(6기), 산포지구(6기), 직산지구(6기)가 원전 예정지역으로 지정되면서, 먼저 북면 부구리 지역에 원전 6기가 건설되었습니다.\n',
 '해안선이 잘려나가고 마을 한복판에 고압 송전탑이 들어섰습니다.\n',
 '어장이 파괴되고 지역 특산품에 방사능 꼬리표가 붙었습니다.\n',
 '삶의 터전이 무너졌으며 선량한 울진군민은 원전반대 시위를 했다는 이유로 범죄자가 되었습니다.\n',
 '그러던 지난 1999년 정부는 산포지구에 전원개발사업예정구역 지정고시 강행 통보를 통해 추가원전을 건설하려 했습니다.\n',
 '저를 비롯한 모든 지역주민이 결사반대 투쟁을 했지만 정부는 일방적이었습니다.\n',
 '울진군민과 정부(산업부 장관, 한수원 사장)의 오랜 협의결과, 산포지구(6기), 직산지구(6기)를 해제하는 조건으로 ‘신규원전 4개호기 건설을 마지막으로 더 이상의 원전 관련 시설을 건설하지 않겠다’는 약속을 함으로써, 북면 부구리 기존 지구에 4기의 추가 원전건설을 수용했습니다.\n',
 '그것이 바로 신한울 1,2,3,4호기였고 정부

In [19]:
with open("data/petitions.txt", 'w') as f:
    f.writelines(petitions)

In [6]:
total_morphs = []

for sentence in tqdm(petitions):
    if len(sentence) > 0:
        morph_sentence = []

        for word in sentence.split(" "):
            count = 0
            for token, pos in mecab.pos(word):
                if count > 0:
                    if not pos.startswith("N"):
                        token = "##" + token
                        morph_sentence.append(token)
                    else:
                        morph_sentence.append(token)
                else:
                    morph_sentence.append(token)
                    count += 1
        morph_sentence[-1] = morph_sentence[-1] + "\n"

        total_morphs.append(morph_sentence)

output = [" ".join(morphs) for morphs in total_morphs]
# total_morphs = list(itertools.chain.from_iterable(total_morphs))

100%|██████████| 155357/155357 [00:32<00:00, 4763.26it/s]


In [7]:
with open("data/petition_proc_with_hash_code.txt", 'w') as f:
    f.writelines(output)

In [8]:
from tokenizers import BertWordPieceTokenizer, SentencePieceBPETokenizer, CharBPETokenizer, ByteLevelBPETokenizer

how_to_tokenize = BertWordPieceTokenizer

if str(how_to_tokenize) == str(BertWordPieceTokenizer):
    print('BertWordPieceTokenizer')
    tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
elif str(how_to_tokenize) == str(SentencePieceBPETokenizer):
    print('SentencePieceBPETokenizer')
    tokenizer = SentencePieceBPETokenizer()
elif str(how_to_tokenize) == str(CharBPETokenizer):
    print('CharBPETokenizer')
    tokenizer = CharBPETokenizer() 
elif str(how_to_tokenize) == str(ByteLevelBPETokenizer):
    print('ByteLevelBPETokenizer')
    tokenizer = ByteLevelBPETokenizer()
else:
    assert('select right tokenizer')

corpus_file    = ['data/petition_proc_with_hash_code.txt']  # data path
vocab_size     = 32000
limit_alphabet = 6000
output_path    = 'hugging_%d'%(vocab_size)
min_frequency  = 5

# Then train it!
tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet,
               show_progress=True)

BertWordPieceTokenizer


In [9]:
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

sentence = 'I want to go my hometown'
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

나는 오늘 아침밥을 먹었다.
=>idx   : [1297, 3608, 5316, 29923, 3532, 1813, 3684, 3393, 18]
=>tokens: ['나', '##는', '오늘', '아침밥', '##을', '먹', '##었', '##다', '.']
=>offset: [(0, 1), (1, 2), (3, 5), (6, 9), (9, 10), (11, 12), (12, 13), (13, 14), (14, 15)]
=>decode: 나는 오늘 아침밥을 먹었다.

I want to go my hometown
=>idx   : [45, 91, 20128, 8501, 8297, 26788, 26784, 19487, 27697]
=>tokens: ['I', 'w', '##ant', 'to', 'go', 'my', 'home', '##to', '##wn']
=>offset: [(0, 1), (2, 3), (3, 6), (7, 9), (10, 12), (13, 15), (16, 20), (20, 22), (22, 24)]
=>decode: I want to go my hometown



In [10]:
hf_model_path='data/tokenizer_model'

if not os.path.isdir(hf_model_path):
    os.mkdir(hf_model_path)

tokenizer.save_model(hf_model_path)

['data/tokenizer_model/vocab.txt']

In [11]:
from transformers import BertTokenizerFast

tokenizer_for_load = BertTokenizerFast.from_pretrained(hf_model_path,
                                                       strip_accents=False,
                                                       lowercase=False)

print('vocab size : %d' % tokenizer_for_load.vocab_size)
tokenized_input_for_pytorch = tokenizer_for_load("나는 오늘 아침밥을 먹었다.", return_tensors="pt")

print("Tokens (str)      : {}".format([tokenizer_for_load.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))

vocab size : 32000
Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아침밥', '##을', '먹', '##었', '##다', '.', '[SEP]']
Tokens (int)      : [2, 1297, 3608, 5316, 29923, 3532, 1813, 3684, 3393, 18, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [12]:
tokenizer_for_load.all_special_tokens # 추가하기 전 기본적인 special token

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']